In [1]:
import pandas as pd
import json
from openai import OpenAI

# Load the CSV file
batches_to_keys = pd.read_csv("batches_OpenAI.csv")

In [3]:
# Iterate through each row in the CSV file
all_categories = []  # To collect all category data

for _, row in batches_to_keys.iterrows():
    batch_job_id = row['batch_job_id']
    api_key = row['api_key']
    
    # Create the OpenAI client
    client = OpenAI(api_key=api_key)
    
    try:
        # Get the results file ID for the batch job (assuming this comes from an API or prior step)
        result_file_id = client.batch_jobs.retrieve(batch_job_id).output_file_id
        
        # Download the results file
        result_content = client.files.content(result_file_id).content
        result_file_name = "batch_results.jsonl"
        with open(result_file_name, 'wb') as file:
            file.write(result_content)  # Save the content to a file

        # Parse the results
        results = []
        with open(result_file_name, 'r') as file:
            for line in file:
                results.append(json.loads(line.strip()))
        results = pd.DataFrame(results)

        # Extract the sentences for each topic
        def extract_sentences(response):
            try:
                # Parse the assistant's message content
                content = json.loads(response['body']['choices'][0]['message']['content'])
                return {
                    "food_sentences": " ".join(content.get("food_sentences", [])),
                    "service_sentences": " ".join(content.get("service_sentences", [])),
                    "atmosphere_sentences": " ".join(content.get("atmosphere_sentences", [])),
                    "price_sentences": " ".join(content.get("price_sentences", [])),
                }
            except Exception as e:
                print(f"Error parsing response: {e}")
                return {
                    "food_sentences": None,
                    "service_sentences": None,
                    "atmosphere_sentences": None,
                    "price_sentences": None,
                }

        # Apply the extraction to the 'response' column
        category_data = results['response'].apply(extract_sentences)

        # Create a new DataFrame with the extracted category sentences
        category_df = pd.DataFrame(category_data.tolist())

        # Combine 'custom_id' and extracted category sentences
        category_df['custom_id'] = results['custom_id']
        # Rename custom_id to review_id
        category_df.rename(columns={"custom_id": "review_id"}, inplace=True)

        # Append the results for this batch to the collection
        all_categories.append(category_df)

    except Exception as e:
        print(f"Error processing batch_job_id {batch_job_id}: {e}")

# Combine all results into a single DataFrame
final_df = pd.concat(all_categories, ignore_index=True)

# Save the final DataFrame to a CSV file
final_df.to_csv("extracted_category_data.csv", index=False)
print("Processing complete. Results saved to 'extracted_category_data.csv'.")


Error processing batch_job_id  batch_6796154b62a481909fbda982a027349a: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id  batch_6796154e85708190a00827696e39cd50: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id  batch_67961eb62c5c8190ba132744e53eef87: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id   batch_67961eb8f0bc81909061266ddec5ff0f: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id batch_6796270cd7c48190b524d5e3539a5437: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id batch_6796270f8f9481909efa4fa8cc803c35: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id batch_679627d54b808190bdca2838cf87585c: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id batch_679627d7ee5481909f8ae5c1e07381fe: 'OpenAI' object has no attribute 'batch_jobs'
Error processing batch_job_id batch_67962a9f9b2481908238f7ac692728a

ValueError: No objects to concatenate